# Book Recommendation System | Colaborative filtering | Clustering

In [61]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns

## Data Cleansing and Preparation

In [62]:
books = pd.read_csv("../data/BX-Books.csv", sep=";", on_bad_lines="skip", encoding="latin-1")

C:\Users\levyl\AppData\Local\Temp\ipykernel_30836\279171129.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv("../data/BX-Books.csv", sep=";", on_bad_lines="skip", encoding="latin-1")


In [63]:
books.head(3)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...


In [64]:
books.shape

(271360, 8)

In [65]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [105]:
books.describe()

,ISBN,title,author,Year-Of-Publication,publisher,img_url
count,271360,271360,271358,271360,271358,271357
unique,271360,242135,102022,202,16807,271041
top,020130998X,Selected Poems,Agatha Christie,2002,Harlequin,http://images.amazon.com/images/P/155936078X.0...
freq,1,27,632,13903,7535,2


In [66]:
books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher','Image-URL-L']]

In [67]:
books.head(3)

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...


In [68]:
books.rename(columns={"Book-Title": "title",
                      "Book-Author": "author",
                      "Year-Of-Publisher": "year",
                      "Publisher" : "publisher",
                      "Image-URL-L" : "img_url"}, inplace = True)

In [69]:
books.head(3)

,ISBN,title,author,Year-Of-Publication,publisher,img_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...


In [70]:
users = pd.read_csv("../data/BX-Users.csv", sep=";", on_bad_lines="skip", encoding="latin-1")

In [71]:
users.shape

(278858, 3)

In [72]:
users.head(3)

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN


In [106]:
users.describe()

,User-ID,Age
count,278858.00000,168096.000000
mean,139429.50000,34.751434
std,80499.51502,14.428097
min,1.00000,0.000000
25%,69715.25000,24.000000
50%,139429.50000,32.000000
75%,209143.75000,44.000000
max,278858.00000,244.000000


In [73]:
ratings = pd.read_csv("../data/BX-Book-Ratings.csv", sep=";", on_bad_lines="skip", encoding="latin-1")

In [74]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [107]:
ratings.describe()

,user_id,rating
count,1.149780e+06,1.149780e+06
mean,1.403864e+05,2.866950e+00
std,8.056228e+04,3.854184e+00
min,2.000000e+00,0.000000e+00
25%,7.034500e+04,0.000000e+00
50%,1.410100e+05,0.000000e+00
75%,2.110280e+05,7.000000e+00
max,2.788540e+05,1.000000e+01


In [75]:
ratings.shape

(1149780, 3)

In [76]:
print(books.shape)

(271360, 6)


In [77]:
print(users.shape)

(278858, 3)


In [79]:
ratings.rename(columns={
    "User-ID":"user_id",
    "Book-Rating": "rating"}, inplace=True)

In [80]:
ratings.head()

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [82]:
ratings["user_id"].value_counts()

user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
119573        1
276706        1
276697        1
276679        1
276676        1
Name: count, Length: 105283, dtype: int64

In [83]:
ratings["user_id"].unique().shape

(105283,)

In [85]:
x = ratings["user_id"].value_counts() > 200

In [90]:
x

user_id
11676      True
198711     True
153662     True
98391      True
35859      True
          ...  
119573    False
276706    False
276697    False
276679    False
276676    False
Name: count, Length: 105283, dtype: bool

In [92]:
x[x].shape

(899,)

In [93]:
y = x[x].index

In [94]:
y

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       116122,  44296,  28634,  59727,  73681, 274808, 188951,   9856, 155916,
       268622],
      dtype='int64', name='user_id', length=899)

In [98]:
ratings_clean = ratings[ratings["user_id"].isin(y)]

In [99]:
ratings_clean.head()

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [100]:
ratings_clean.shape

(526356, 3)

In [108]:
ratings_clean.describe()

,user_id,rating
count,526356.000000,526356.000000
mean,141855.893327,1.969448
std,80616.791840,3.507206
min,254.000000,0.000000
25%,73394.000000,0.000000
50%,144131.000000,0.000000
75%,212898.000000,3.000000
max,278418.000000,10.000000


In [101]:
ratings_with_books = ratings_clean.merge(books, on = "ISBN")

In [102]:
ratings_with_books.head()

,user_id,ISBN,rating,title,author,Year-Of-Publication,publisher,img_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...
2,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning,http://images.amazon.com/images/P/003008685X.0...
3,277427,0030615321,0,"Ask for May, Settle for June (A Doonesbury book)",G. B. Trudeau,1982,Henry Holt &amp; Co,http://images.amazon.com/images/P/0030615321.0...
4,277427,0060002050,0,On a Wicked Dawn (Cynster Novels),Stephanie Laurens,2002,Avon Books,http://images.amazon.com/images/P/0060002050.0...


In [103]:
ratings_with_books.shape

(487671, 8)

In [109]:
num_rating = ratings_with_books.groupby("title")["rating"].count().reset_index()

In [111]:
num_rating.head(3)

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1


In [112]:
num_rating.rename(columns={"rating": "num_of_rating"}, inplace=True)

In [113]:
final_rating = ratings_with_books.merge(num_rating, on = "title")

In [114]:
final_rating.head()

,user_id,ISBN,rating,title,author,Year-Of-Publication,publisher,img_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...,7
2,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning,http://images.amazon.com/images/P/003008685X.0...,1
3,277427,0030615321,0,"Ask for May, Settle for June (A Doonesbury book)",G. B. Trudeau,1982,Henry Holt &amp; Co,http://images.amazon.com/images/P/0030615321.0...,1
4,277427,0060002050,0,On a Wicked Dawn (Cynster Novels),Stephanie Laurens,2002,Avon Books,http://images.amazon.com/images/P/0060002050.0...,13


In [115]:
final_rating.shape

(487671, 9)

In [117]:
final_rating = final_rating[final_rating["num_of_rating"]>= 50]

In [118]:
final_rating.head()

,user_id,ISBN,rating,title,author,Year-Of-Publication,publisher,img_url,num_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
13,277427,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,133
15,277427,0060934417,0,Bel Canto: A Novel,Ann Patchett,2002,Perennial,http://images.amazon.com/images/P/0060934417.0...,108
18,277427,0061009059,9,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich,1995,HarperTorch,http://images.amazon.com/images/P/0061009059.0...,108
24,277427,006440188X,0,The Secret Garden,Frances Hodgson Burnett,1998,HarperTrophy,http://images.amazon.com/images/P/006440188X.0...,79


In [120]:
final_rating.sample(10)

,user_id,ISBN,rating,title,author,Year-Of-Publication,publisher,img_url,num_of_rating
435733,245645,0312924801,10,September,Rosamunde Pilcher,1991,St. Martin's Press,http://images.amazon.com/images/P/0312924801.0...,50
176216,101209,0312924585,0,Silence of the Lambs,Thomas Harris,1991,St. Martin's Press,http://images.amazon.com/images/P/0312924585.0...,113
346892,200226,0553268880,0,The Prince of Tides,Pat Conroy,1987,Bantam Books,http://images.amazon.com/images/P/0553268880.0...,148
189144,109574,0451142934,8,Skeleton Crew,Stephen King,1986,New Amer Library,http://images.amazon.com/images/P/0451142934.0...,82
426537,239584,0385475721,7,The Blind Assassin,Margaret Atwood,2000,Nan A. Talese,http://images.amazon.com/images/P/0385475721.0...,73
383701,224349,0743237188,0,Fall On Your Knees (Oprah #45),Ann-Marie MacDonald,2002,Touchstone,http://images.amazon.com/images/P/0743237188.0...,90
130318,76352,0446604275,0,Primary Colors: A Novel of Politics,Anonymous,1996,Warner Books,http://images.amazon.com/images/P/0446604275.0...,68
405981,232131,0373825013,0,"Whirlwind (Tyler, Book 1)",Nancy Martin,1992,Harlequin,http://images.amazon.com/images/P/0373825013.0...,78
110859,60244,0380789019,0,Neverwhere,Neil Gaiman,1998,Avon,http://images.amazon.com/images/P/0380789019.0...,72
302554,175003,0440213290,0,The Copper Beech,Maeve Binchy,1993,Dell,http://images.amazon.com/images/P/0440213290.0...,68


In [121]:
final_rating.shape

(61853, 9)

In [122]:
final_rating.drop_duplicates(["user_id", "title"], inplace = True)

In [123]:
final_rating.shape

(59850, 9)

## Creating the Embedding Matrix

In [125]:
book_pivot = final_rating.pivot_table(columns="user_id", index="title", values="rating" )

In [126]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [127]:
book_pivot.shape

(742, 888)

In [129]:
book_pivot.fillna(0, inplace=True)

In [131]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [133]:
from scipy.sparse import csr_matrix

In [134]:
book_sparse = csr_matrix(book_pivot)

In [135]:
book_sparse

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 14961 stored elements and shape (742, 888)>

## Model Training


In [138]:
from sklearn.neighbors import NearestNeighbors

In [140]:
model = NearestNeighbors(algorithm="brute")

In [141]:
model.fit(book_sparse)

NearestNeighbors(algorithm='brute')

In [143]:
distance, suggestion = model.kneighbors(book_pivot.iloc[237, :].values.reshape(1, -1), n_neighbors=6)

In [144]:
distance

array([[ 0.        , 67.75691847, 68.05145112, 72.277244  , 75.81556568,
        76.30203143]])

In [145]:
suggestion

array([[237, 238, 240, 241, 184, 536]])

In [146]:
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])


Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


In [147]:
book_pivot.index[3]

'4 Blondes'

In [150]:
book_names = book_pivot.index

In [151]:
book_names

Index(['1984', '1st to Die: A Novel', '2nd Chance', '4 Blondes',
       '84 Charing Cross Road', 'A Bend in the Road', 'A Case of Need',
       'A Child Called \It\": One Child's Courage to Survive"',
       'A Civil Action', 'A Cry In The Night',
       ...
       'Winter Solstice', 'Wish You Well', 'Without Remorse',
       'Wizard and Glass (The Dark Tower, Book 4)', 'Wuthering Heights',
       'Year of Wonders', 'You Belong To Me',
       'Zen and the Art of Motorcycle Maintenance: An Inquiry into Values',
       'Zoya', '\O\" Is for Outlaw"'],
      dtype='object', name='title', length=742)

In [152]:
import pickle
pickle.dump(model, open("../artifacts/model.pkl", "wb"))
pickle.dump(book_names, open("../artifacts/book_names.pkl", "wb"))
pickle.dump(final_rating, open("../artifacts/final_ratings.pkl", "wb"))
pickle.dump(book_pivot, open("../artifacts/book_pivot.pkl", "wb"))

## Create Web App

In [162]:
def recommend_book(book_name):
    book_id  = np.where(book_pivot.index == book_name)[0][0]
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id, :].values.reshape(1, -1), n_neighbors=6)

    for i in range(len(suggestion)):
        books = book_pivot.index[suggestion[i]]
        for j in books:
            print(j)

In [170]:
book_name = '1984'

In [173]:
recommend_book(book_name)

1984
No Safe Place
Long After Midnight
A Civil Action
Abduction
Master of the Game
